Testing Generator

In [13]:
import anthropic
import os 
from dotenv import load_dotenv, dotenv_values

load_dotenv() 
api_key = os.getenv("ANTHROPIC_KEY")

client = anthropic.Anthropic(
    api_key=api_key,
)

def question_generator(articles:dict,
                       style: str,
                       prompt: str = "ministerial interview"):
        message = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1000,
        temperature=0,
        system="""
        You are a journalist crafting a single challenging question for a press conference with a government minister.
            
            Your question should be direct, provocative, and concise. Only include the question and not context for the prompt.

            
            """,  # <-- system prompt
        messages=[
            {
                "role": "user",
                
                "content": [
                    {
                        "type": "text",
                        "text": f'Based on the articles provided: {articles}, craft a question in the style of a {style} journalist in the context of {prompt}.   DO NOT INCLUDE THE NAME OR TITLE OF THE PERSON YOU ARE INTERVIEWING. The questioner may not be the subject of the news story.'

                    }
                ]
            }
        ]
    )
        return message.content


In [16]:
output = question_generator("testing","happy")

In [20]:
output[0].text

list

News Catcher

In [153]:
import anthropic
import pandas as pd
import os
import re
from pprint import pprint
from newscatcherapi_client import Newscatcher, ApiException
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd 


from dotenv import load_dotenv, dotenv_values


load_dotenv() 
claude_api_key = os.getenv("ANTHROPIC_KEY")
news_catcher_api_key = os.getenv("NEWS_CATCHER_API")


newscatcher = Newscatcher(api_key = news_catcher_api_key)

client = anthropic.Anthropic(api_key = claude_api_key)


model = ChatAnthropic(model_name='claude-3-opus-20240229', temperature=0.6, api_key=claude_api_key)
def generate_terms(topic = "policing"):
    total_prompt = f"""Generate 5 diverse single word search terms for news articles based on this briefing {topic}
    Consider the you will be taking this information to brief the priminister, so take the most important infromation.
    Only return a python list of keywords.
    """
    output = model.invoke(total_prompt)
    terms = extract_bracket_contents(output.content)
    return terms
def extract_bracket_contents(text):
    pattern = r'\[(.*?)\]'  # Pattern to find text inside brackets
    list_of_terms  = re.findall(pattern, text)[0].split(",")
    return [text.strip(" '") for text in list_of_terms]

def create_newscatcher_query(terms):
    query_result = "("
    for word in terms:
        if word == terms[-1]:
            query_result += "(" + word + ")) AND Britain"
        else:
            query_result += "(" + word + ")" + " OR "
    return query_result

def query_newscatcher(query=""):
    if not query :
        return newscatcher.search.get(q="Britain",
                                      search_in="content",
                                      lang = "en",
                                      theme = "Politics",
                                      predefined_sources = "top 30 GB",
                                      from_ = "2 days ago",
                                      clustering_enabled = True,
                                      page_size = 1000,
                                      include_nlp_data = True)
    else:
        return newscatcher.search.get(q=query,
                                      search_in="content",
                                      lang = "en",
                                      predefined_sources = "top 100 GB",
                                      from_ = "7 days ago",
                                      clustering_enabled = True,
                                      page_size = 1000,
                                      include_nlp_data = True)


def extract_summary_texts(newscatcher_output):
    summary_list = []
    cluster_list = [x["articles"] for x in newscatcher_output.dict()["clusters"]][:5]
    for cluster in cluster_list:
        summary_text=""
        for article in cluster:
            summary_text += " ``` " + article["nlp"]["summary"]
        summary_list.append(summary_text)
        
    return summary_list   


def generate_summary(summary_list):
    succinct_summary = []
    for summary in summary_list:
        message = client.messages.create(
            model="claude-3-opus-20240229",
            max_tokens=1000,
            temperature=0.4,
            messages=[
                {
                    "role": "user",
                    
                    "content": [
                        {
                            "type": "text",
                            "text": f'Summarise the following content separated by ``` into one sentance: {summary}.'
                            
                        }
                    ]
                }
            ]
        )
        succinct_summary.append(message.content)
    return succinct_summary 



def newscatcher_main(query):
    test_terms = generate_terms(query)
    test_query = create_newscatcher_query(test_terms)
    test_nc_output = query_newscatcher(test_query)
    test_summary_text = extract_summary_texts(test_nc_output)
    test_output = generate_summary(test_summary_text)
    return test_output


In [154]:
string = "Prepare me for a briefing that will be given by the primeminister on speech on policing in the north east of england"

In [142]:
test_terms = generate_terms(string)
test_query = create_newscatcher_query(test_terms)

In [143]:
test_terms

['crime', 'enforcement', 'community', 'budget', 'strategy']

In [151]:
test_nc_output = query_newscatcher(test_query)

In [155]:
newcatcher_output = newscatcher_main(string)

In [157]:
newcatcher_output

[[TextBlock(text='Liz Truss, who served as Prime Minister for just 49 days, has been accused by Labour\'s deputy leader Angela Rayner of staging a "twisted victory lap" while promoting her new book "Ten Years To Save The West," despite mortgage repossession claims soaring following her disastrous mini-budget.', type='text')],
 [TextBlock(text="Katharine Birbalsingh, the headteacher of Michaela Community School in Brent, London, won a High Court case against a Muslim student who challenged the school's ban on prayer rituals, which was introduced due to death and bomb threats linked to religious observance on site.", type='text')],
 [TextBlock(text='Sir Keir Starmer, the leader of the Labour Party, has committed to increasing the UK\'s defence budget to 2.5% of GDP and implementing a "triple lock" on the country\'s nuclear deterrent, which includes building four new Dreadnought submarines in Barrow-in-Furness, despite opposition from the SNP and anti-nuclear groups.', type='text')],
 [Te

In [98]:
type(test_nc_output)

newscatcherapi_client.pydantic.clustering_search_response.ClusteringSearchResponse

In [76]:
test_nc_output.clusters[3].articles[1]['title']

"JK Rowling 'won't forgive' Daniel Radcliffe and Emma Watson"

In [159]:
top_5_articles = [newcatcher_output[0][0].text,
newcatcher_output[1][0].text,
newcatcher_output[2][0].text,
newcatcher_output[3][0].text,
newcatcher_output[4][0].text]
top_5_articles

['Liz Truss, who served as Prime Minister for just 49 days, has been accused by Labour\'s deputy leader Angela Rayner of staging a "twisted victory lap" while promoting her new book "Ten Years To Save The West," despite mortgage repossession claims soaring following her disastrous mini-budget.',
 "Katharine Birbalsingh, the headteacher of Michaela Community School in Brent, London, won a High Court case against a Muslim student who challenged the school's ban on prayer rituals, which was introduced due to death and bomb threats linked to religious observance on site.",
 'Sir Keir Starmer, the leader of the Labour Party, has committed to increasing the UK\'s defence budget to 2.5% of GDP and implementing a "triple lock" on the country\'s nuclear deterrent, which includes building four new Dreadnought submarines in Barrow-in-Furness, despite opposition from the SNP and anti-nuclear groups.',
 'Here are the one-sentence summaries for each content block separated by ```:\n\n1. Jeremy Hunt 

In [ ]:
newcatcher_output

In [100]:
import anthropic
import os 
from dotenv import load_dotenv, dotenv_values

load_dotenv() 
api_key = os.getenv("ANTHROPIC_KEY")

client = anthropic.Anthropic(
    api_key=api_key,
)

def get_sentiment(articles:dict):
        message = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1000,
        temperature=0,
        system="""

            
            """,  # <-- system prompt
        messages=[
            {
                "role": "user",
                
                "content": [
                    {
                        "type": "text",
                        "text": f'Provide me the general sentiment on these articles here: {articles}, in less than 10 words'

                    }
                ]
            }
        ]
    )
        return message.content


Test polling

In [167]:
import sys  
sys.path.insert(1,'/Users/mawuliagamah/gitprojects/octelligence/polling')

from insights import get_insights
from audience_id import get_audiences

FileNotFoundError: [Errno 2] No such file or directory: 'polling/data/polling.csv'

In [164]:
get_audiences()

NameError: name 'get_audiences' is not defined